In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

Study population

In [3]:
data_dir = '../data/'
df_demo = pd.DataFrame(pd.read_excel(data_dir + 'CHEXPERT_DEMO.xlsx', engine='openpyxl'))

df_demo = df_demo.rename(columns={'PRIMARY_RACE': 'race'})
df_demo = df_demo.rename(columns={'PATIENT': 'patient_id'})
df_demo = df_demo.rename(columns={'GENDER': 'sex'})
df_demo = df_demo.rename(columns={'AGE_AT_CXR': 'age'})
df_demo = df_demo.rename(columns={'ETHNICITY': 'ethnicity'})
df_demo = df_demo.drop(['sex', 'age'], axis=1)
print("df demo data")
print(df_demo.shape)
df_demo.head()

df demo data
(65401, 3)


,patient_id,race,ethnicity
0,patient24428,White,Non-Hispanic/Non-Latino
1,patient48289,Other,Hispanic/Latino
2,patient33856,White,Non-Hispanic/Non-Latino
3,patient41673,Unknown,Unknown
4,patient48493,White,Non-Hispanic/Non-Latino


In [3]:
df_img_data = pd.read_csv(data_dir + 'train_cheXbert.csv')
print(df_img_data.shape)
df_img_data.head()

(223414, 19)


,Path,Sex,Age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,NaN,1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,NaN
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [4]:
df_data_split = pd.read_csv(data_dir + 'chexpert_split_2021_08_20.csv').set_index('index')
print(df_data_split.shape)
df_data_split.head()

(127118, 2)


,Unnamed: 0,split
index,,
1,0,train
2,1,train
4,2,train
5,3,train
7,4,train


In [5]:
df_img_data = pd.concat([df_img_data,df_data_split], axis=1)
df_img_data = df_img_data[~df_img_data.split.isna()]
print(df_img_data.shape)
df_img_data.head()


(127118, 21)


,Path,Sex,Age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,...,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,Unnamed: 0,split
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,NaN,1.0,1.0,NaN,-1.0,...,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,NaN,0.0,train
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,train
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.0,train
5,CheXpert-v1.0/train/patient00004/study1/view1_...,Female,20,Frontal,PA,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,3.0,train
7,CheXpert-v1.0/train/patient00005/study1/view1_...,Male,33,Frontal,PA,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,1.0,1.0,4.0,train


In [6]:
split =  df_img_data.Path.str.split("/", expand = True)
print(split.shape)
split.head()

(127118, 5)


,0,1,2,3,4
1,CheXpert-v1.0,train,patient00002,study2,view1_frontal.jpg
2,CheXpert-v1.0,train,patient00002,study1,view1_frontal.jpg
4,CheXpert-v1.0,train,patient00003,study1,view1_frontal.jpg
5,CheXpert-v1.0,train,patient00004,study1,view1_frontal.jpg
7,CheXpert-v1.0,train,patient00005,study1,view1_frontal.jpg


In [7]:
df_img_data["patient_id"] = split[2]
df_img_data = df_img_data.rename(columns={'Age': 'age'})
df_img_data = df_img_data.rename(columns={'Sex': 'sex'})
print(df_img_data.shape)
df_img_data.head()


(127118, 22)


,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,...,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,Unnamed: 0,split,patient_id
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,NaN,1.0,1.0,NaN,-1.0,...,-1.0,NaN,-1.0,NaN,1.0,NaN,NaN,0.0,train,patient00002
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,train,patient00002
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,1.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,2.0,train,patient00003
5,CheXpert-v1.0/train/patient00004/study1/view1_...,Female,20,Frontal,PA,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,3.0,train,patient00004
7,CheXpert-v1.0/train/patient00005/study1/view1_...,Male,33,Frontal,PA,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,1.0,1.0,4.0,train,patient00005


In [8]:
df_cxr = df_demo.merge(df_img_data, on="patient_id")
df_cxr.head()
print(df_cxr.shape)
df_cxr.head()

(127118, 24)


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,Unnamed: 0,split
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,59,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,53688.0,train
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,61,Frontal,PA,NaN,NaN,...,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,53689.0,train
2,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,53690.0,train
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,60,Frontal,AP,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,53691.0,train
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study45/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,53692.0,train


In [9]:
df_cxr.race.value_counts()

race
White                        83069
Asian                        16911
White, non-Hispanic          15951
Black or African American     7941
Asian, non-Hispanic           1911
Black, non-Hispanic           1320
Asian - Historical Conv          8
White or Caucasian               7
Name: count, dtype: int64

In [10]:
white = 'White'
asian = 'Asian'
black = 'Black'

In [11]:
mask = (df_cxr.race.str.contains("Black", na=False))
df_cxr.loc[mask, "race"] = black

mask = (df_cxr.race.str.contains("White", na=False))
df_cxr.loc[mask, "race"] = white

mask = (df_cxr.race.str.contains("Asian", na=False))
df_cxr.loc[mask, "race"] = asian

df_cxr['race'].unique()
print(df_cxr.shape)

(127118, 24)


In [12]:
df_cxr.race.value_counts()

race
White    99027
Asian    18830
Black     9261
Name: count, dtype: int64

In [13]:
df_cxr = df_cxr[df_cxr.race.isin([asian,black,white])]
print(df_cxr.shape)

(127118, 24)


In [14]:
df_cxr = df_cxr[df_cxr.ethnicity.isin(["Non-Hispanic/Non-Latino","Not Hispanic"])]
print(df_cxr.shape)

(127118, 24)


In [15]:
df_cxr = df_cxr[df_cxr["Frontal/Lateral"]=="Frontal"]
print(df_cxr.shape)

(127118, 24)


In [16]:
df_cxr['race_label'] = df_cxr['race']

df_cxr.loc[df_cxr['race_label'] == white, 'race_label'] = 0
df_cxr.loc[df_cxr['race_label'] == asian, 'race_label'] = 1
df_cxr.loc[df_cxr['race_label'] == black, 'race_label'] = 2
print(df_cxr.shape)

(127118, 25)


In [17]:
df_cxr['sex_label'] = df_cxr['sex']

df_cxr.loc[df_cxr['sex_label'] == 'Male', 'sex_label'] = 0
df_cxr.loc[df_cxr['sex_label'] == 'Female', 'sex_label'] = 1
print(df_cxr.shape)

(127118, 26)


In [18]:
labels = [
    'No Finding',
    'Enlarged Cardiomediastinum',
    'Cardiomegaly',
    'Lung Opacity',
    'Lung Lesion',
    'Edema',
    'Consolidation',
    'Pneumonia',
    'Atelectasis',
    'Pneumothorax',
    'Pleural Effusion',
    'Pleural Other',
    'Fracture',
    'Support Devices']

In [19]:
df_cxr['disease'] = df_cxr[labels[0]]
df_cxr.loc[df_cxr[labels[0]] == 1, 'disease'] = labels[0]
df_cxr.loc[df_cxr[labels[10]] == 1, 'disease'] = labels[10]
df_cxr.loc[df_cxr['disease'].isna(), 'disease'] = 'Other'
print(df_cxr.shape)
df_cxr['disease_label'] = df_cxr['disease']
df_cxr.loc[df_cxr['disease_label'] == labels[0], 'disease_label'] = 0
df_cxr.loc[df_cxr['disease_label'] == labels[10], 'disease_label'] = 1
df_cxr.loc[df_cxr['disease_label'] == 'Other', 'disease_label'] = 2
print(df_cxr.shape)
df_cxr.head()

(127118, 27)
(127118, 28)


/tmp/ipykernel_30763/2995295540.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'No Finding' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_cxr.loc[df_cxr[labels[0]] == 1, 'disease'] = labels[0]


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Pleural Other,Fracture,Support Devices,No Finding,Unnamed: 0,split,race_label,sex_label,disease,disease_label
0,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study22/view1...,Male,59,Frontal,AP,NaN,NaN,...,NaN,NaN,1.0,NaN,53688.0,train,0,0,Other,2
1,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study39/view1...,Male,61,Frontal,PA,NaN,NaN,...,NaN,NaN,NaN,NaN,53689.0,train,0,0,Other,2
2,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study61/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,1.0,NaN,53690.0,train,0,0,Pleural Effusion,1
3,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study35/view1...,Male,60,Frontal,AP,NaN,NaN,...,NaN,NaN,1.0,NaN,53691.0,train,0,0,Other,2
4,patient24428,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient24428/study45/view1...,Male,61,Frontal,AP,NaN,NaN,...,NaN,NaN,1.0,NaN,53692.0,train,0,0,Pleural Effusion,1


In [20]:
df_cxr.race.value_counts()

race
White    99027
Asian    18830
Black     9261
Name: count, dtype: int64

In [21]:
df_cxr = df_cxr.sort_values(by=['Path'])
df_cxr = df_cxr.reset_index()
df_cxr = df_cxr.drop(columns=['index'])
df_cxr = df_cxr[df_cxr["AP/PA"].isin(['AP','PA'])]
print(df_cxr.shape)
df_cxr.head()

(127118, 28)


,patient_id,race,ethnicity,Path,sex,age,Frontal/Lateral,AP/PA,Enlarged Cardiomediastinum,Cardiomegaly,...,Pleural Other,Fracture,Support Devices,No Finding,Unnamed: 0,split,race_label,sex_label,disease,disease_label
0,patient00002,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,NaN,NaN,...,NaN,1.0,NaN,NaN,1.0,train,0,1,Other,2
1,patient00002,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,NaN,1.0,...,NaN,1.0,NaN,NaN,0.0,train,0,1,Other,2
2,patient00003,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,train,0,0,Other,2
3,patient00004,Black,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient00004/study1/view1_...,Female,20,Frontal,PA,0.0,NaN,...,NaN,NaN,NaN,1.0,3.0,train,2,1,No Finding,0
4,patient00005,White,Non-Hispanic/Non-Latino,CheXpert-v1.0/train/patient00005/study1/view1_...,Male,33,Frontal,PA,1.0,0.0,...,NaN,NaN,1.0,1.0,4.0,train,0,0,No Finding,0
